In [1]:
import os
import time
import torch
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

In [2]:
batch_size = 60
epochs = 5
seed = 1
log_interval = 10
torch.manual_seed(seed)

In [3]:
train_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(r"C:\Users\shiva\Kuzushiji-Kanji\kanjivgmain2", transform=transforms.ToTensor()), batch_size=batch_size)

In [5]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(4096, 2000)
        self.fc2 = nn.Linear(2000, 1000)
        self.fc3 = nn.Linear(1000, 500)
        self.fc41 = nn.Linear(500, 50)
        self.fc42 = nn.Linear(500, 50)
        self.fc5 = nn.Linear(50, 500)
        self.fc6 = nn.Linear(500, 1000)
        self.fc7 = nn.Linear(1000, 2000)
        self.fc8 = nn.Linear(2000, 4096)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        h2 = F.relu(self.fc2(h1))
        h3 = F.relu(self.fc3(h2))
        return self.fc41(h3), self.fc42(h3)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h5 = F.relu(self.fc5(z))
        h6 = F.relu(self.fc6(h5))
        h7 = F.relu(self.fc7(h6))
        return torch.sigmoid(self.fc8(h7))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 4096))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


model = VAE()
optimizer = optim.Adam(model.parameters(), lr=1e-3)


# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 4096), reduction='sum')

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD


def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))

In [ ]:
for epoch in range(1, epochs + 1):
        train(epoch)
        with torch.no_grad():
            sample = torch.randn(40, 50)
            sample = model.decode(sample)
            save_image(sample.view(40, 1, 64, 64),
r"C:\Users\shiva\Kuzushiji-Kanji\sample_" + str(epoch) + '.png')

Train Epoch: 1 [0/11456 (0%)]	Loss: 8523.002083
